# Stylometric analysis of a corpus with genre-diverse works from single authors

Authorship attribution is a use of stylometry which attempts to attribute a disputed text to a single author. Different variations of this technique exist, each with their own pros and cons. A factor which can confuse authorship attribution algorithms is variations in genre within an author's work, which can cause variation in the vocabulary used larger than the variation between authors. This project will explore techniques most effective in a situation like this, testing them against a corpus including assorted works by A. A. Milne, G. K. Chesterton, and George MacDonald, each of whose works cover a variety of genres.

## Techniques using R.stylo

Written in the R language, the `stylo` package includes many tools used for stylometric analysis. The below discussion attempts to discern which of these tools most effectively deals with genre-based noise.

In [ ]:
import rpy2.robjects as ro

R = ro.r

R.library("stylo")

R.setwd(".")

### Cluster Analysis
The following techniques will make use of the `stylo` package's cluster analysis function. The following code runs cluster analysis with the default arguments: 100 MFW, no culling, Burrow's Delta, and no pronouns deleted.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'w', 'ngram.size': 1, 'mfw.min': 100, 'mfw.max': 100, 'delete.pronouns': False, 'analysis.type': 'CA', 'dist.measure': 'dist.delta', 'display.on.screen': True })

This approach exhibits some success; for instance, Chesterton's works of poetry and philosophy are (separately) clustered together. But on closer analysis, it becomes apparent that author's works are only diffentiated from other authors within bubbles of genre; in fact, it is apparent genre has a higher influence on classification than authorship; in fact, Chesterton's work of fiction is included within the cluster of MacDonald's. 
<br>This discrepancy could result on too much attention to genre-specific words. To mitigate this, the following experiment repeats the same parameters, but with only 20 of the most frequent words used, as opposed to 100, and deletes pronouns.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'w', 'ngram.size': 1, 'mfw.min': 20, 'mfw.max': 20, 'delete.pronouns': True, 'analysis.type': 'CA', 'dist.measure': 'dist.delta', 'display.on.screen': True })

Although this approach is not much different, the results differ slightly. For instance, this approach has slightly more success classifying Macdonald's works with each other, but it is still overly distracted by genre noise.<br>
In an attempt to be more independent of genre, the final experiment with cluster analysis will use character bigrams instead of words, with 100 features and pronouns still deleted.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'c', 'ngram.size': 2, 'mfw.min': 100, 'mfw.max': 100, 'delete.pronouns': True, 'analysis.type': 'CA', 'dist.measure': 'dist.delta', 'display.on.screen': True })

This approach yields similar results to the previous ones, but succeeds better in differentiating Milne's plays from Chesterton's poetry.

### Principal Component Analysis
The `R.stylo` package also includes tools for performing stylometric analysis using PCA, or Principal Component Analysis, which may be more flexible for performing some authorship attribution tasks. The following section conducts experiments using this technique. Similarly to the previous section, instructions for repeating experiments will be included with the experimental results.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'w', 'ngram.size': 1, 'mfw.min': 100, 'mfw.max': 100, 'delete.pronouns': False, 'analysis.type': 'PCV', 'text.id.on.graphs': 'labels', 'display.on.screen': True })

Running PCA with the default settings gives similar results to cluster analysis, but it somewhat better shows the differences between Milne's works with the rest of the corpus. Differing the settings from those used with cluster analysis may show more interesting results.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'w', 'ngram.size': 1, 'mfw.min': 20, 'mfw.max': 20, 'culling.min': 50, 'culling.max': 50, 'delete.pronouns': True, 'analysis.type': 'PCV', 'text.id.on.graphs': 'labels', 'display.on.screen': True })

By deleting pronouns, culling at 50%, splitting contractions, and decreasing the number of most frequent words to 20, somewhat better results are acheived. Milne's works are firmly separated from the rest, while Chesterton and MacDonald have their works of fiction and poetry at least marginally separated. Despite these successes, though, Chesterton and MacDonald still have their works broken up by genre, rather than by authorship.

In [ ]:
R.stylo(**{'gui': False, 'corpus.format': 'plain', 'corpus.lang': 'English.all', 'analyzed.features': 'c', 'ngram.size': 2, 'mfw.min': 20, 'mfw.max': 20, 'culling.min': 50, 'culling.max': 50, 'delete.pronouns': True, 'analysis.type': 'PCV', 'text.id.on.graphs': 'labels', 'display.on.screen': True })

A final experiment using the `R.stylo` package, repeating the previous one using character bigrams instead of words, shows only limited usefulness; authorship is vaguely separated from left to right, but the signal still is too weak to be of much use.

## Techniques using scikit_learn
The Python library scikit_learn includes many helpful machine learning libraries which are useful for a variety of natural language processing tasks. Although these rely on a variety of underlying architectures, they are flexible enough to be useful for a wide variety of tasks, including authorship attribution.

### Naive Bayes Classification
Naive Bayes classifiers use a simple derivation of Bayes' Theorem for conditional probability, and rely on a simplifying assumption to vectorize data. By carefully choosing the parameters to such a model, one could abstract away signals in the data which distract from the intended task. Using a Naive Bayes classifier will require vectorizing the data, which is done using a Bag of Words model, which is culled to the 100 most frequent terms. Of each file, the last 10000 words will be used for testing, with the rest used for training. Assembling the test data will be analogous to assembling the training data.

In [ ]:
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

from nltk.tokenize import word_tokenize
from nltk import FreqDist

mipo = open('corpus/milne_pooh.txt')
mipi = open('corpus/milne_pim.txt')
mion = open('corpus/milne_once.txt')
mire = open('corpus/milne_red.txt')
mibe = open('corpus/milne_belinda.txt')

maph = open('corpus/macdonald_phantastes.txt')
mapr = open('corpus/macdonald_princess.txt')
mami = open('corpus/macdonald_miracles.txt')
maun = open('corpus/macdonald_unspoken.txt')

chpo = open('corpus/chesterton_poems.txt')
chwi = open('corpus/chesterton_wisdom.txt')
chgr = open('corpus/chesterton_greybeards.txt')
chev = open('corpus/chesterton_everlasting.txt')
chor = open('corpus/chesterton_orthodoxy.txt')

corpora = [ [ mipo, mipi, mion, mire, mibe ], [ maph, mapr, mami, maun ], [ chpo, chwi, chgr, chev, chor ] ]
corpora_txt = [ [ str([ l + ' ' for l in f ]) for f in c ] for c in corpora ]

def refresh_corpora(v, t, stopwords) :
    corpora_tok = [ [ [ w for w in word_tokenize(f) if w not in stopwords ] for f in c ] for c in corpora_txt ]
    
    corpora_frd_tr = [ [ FreqDist(f[:-t]) for f in c ] for c in corpora_tok ]
    corpora_mfr_tr = [ [ sorted(f.keys(), key=lambda x : f[x], reverse=True)[:v] for f in c ] for c in corpora_frd_tr ]
    corpora_bow_tr = [ [ dict(zip(corpora_mfr_tr[c][f], [ corpora_frd_tr[c][f][x] for x in corpora_mfr_tr[c][f] ])) for f in range(len(corpora_mfr_tr[c])) ] for c in range(len(corpora_mfr_tr)) ]

    corpora_frd_ts = [ [ FreqDist(f[-t:]) for f in c ] for c in corpora_tok ]
    corpora_mfr_ts = [ [ sorted(f.keys(), key=lambda x : f[x], reverse=True)[:v] for f in c ] for c in corpora_frd_ts ]
    corpora_bow_ts = [ [ dict(zip(corpora_mfr_ts[c][f], [ corpora_frd_ts[c][f][x] for x in corpora_mfr_ts[c][f] ])) for f in range(len(corpora_mfr_ts[c])) ] for c in range(len(corpora_mfr_ts)) ]
    
    return (corpora_bow_tr, corpora_mfr_tr, corpora_frd_tr, corpora_bow_ts, corpora_mfr_ts, corpora_frd_ts)

corpora_bow_tr, corpora_mfr_tr, corpora_frd_tr, corpora_bow_ts, corpora_mfr_ts, corpora_frd_ts = refresh_corpora(50, 1000, {})

These bags of words will likely contain a great deal of similar information, but the differences will hopefully prove to be diagnostic.

In [ ]:
print(corpora_bow_tr[0][0])
print(corpora_bow_tr[1][0])
print(corpora_bow_tr[2][0])

One notes that despite many similarities, the sets do, in fact, differ significantly; for instance, that punctuation and "'s" contractions are more common in Milne's corpus, which may denote shorter sentences, while the word 'man' appears in the bags corresponding to both Chesterton and MacDonald, but not Milne. Note that punctuation is included, as this may be indicative of style as well as other types of tokens, for instance by indicating sentence length with the frequency of periods. The next step will be to vectorize this data.

In [ ]:
def refresh_data(cbtr, cmtr, cftr, cbts, cmts, cfts) :
    vocab = list({ s for c in cbtr for b in c for s in b.keys() }.union({ s for c in cbts for b in c for s in b.keys() }))
    
    corpora_bow_tr_i = [ [ dict(zip([vocab.index(ci) for ci in cbtr[c][f]], [ cftr[c][f][x] for x in cmtr[c][f] ])) for f in range(len(cmtr[c])) ] for c in range(len(cmtr)) ]
    X_tr = np.array([ [ corpora_bow_tr_i[c][f][x] if corpora_bow_tr_i[c][f].get(x) else 0 for x in range(len(vocab)) ] for c in range(len(corpora_bow_tr_i)) for f in range(len(corpora_bow_tr_i[c])) ])
    Y_tr = np.array([ c for c in range(len(corpora_bow_tr_i)) for f in range(len(corpora_bow_tr_i[c]))])
    
    corpora_bow_ts_i = [ [ dict(zip([vocab.index(ci) for ci in cbts[c][f]], [ cfts[c][f][x] for x in cmts[c][f] ])) for f in range(len(cmts[c])) ] for c in range(len(cmts)) ]
    X_ts = np.array([ [ corpora_bow_ts_i[c][f][x] if corpora_bow_ts_i[c][f].get(x) else 0 for x in range(len(vocab)) ] for c in range(len(corpora_bow_ts_i)) for f in range(len(corpora_bow_ts_i[c])) ])
    Y_ts = np.array([ c for c in range(len(corpora_bow_ts_i)) for f in range(len(corpora_bow_ts_i[c]))])
    
    return (X_tr, Y_tr, X_ts, Y_ts)

Finally, we can use the assembled training data to train the classifier, then run it on the remaining test data. 

In [ ]:
def test_model(model, v, t, stopwords={}) :
    c_bow_tr, c_mfr_tr, c_frd_tr, c_bow_ts, c_mfr_ts, c_frd_ts = refresh_corpora(v, t, stopwords)
    X_tr, Y_tr, X_ts, Y_ts = refresh_data(c_bow_tr, c_mfr_tr, c_frd_tr, c_bow_ts, c_mfr_ts, c_frd_ts)
    model.fit(X_tr, Y_tr)
    return model.score(X_ts, Y_ts)
test_model(MultinomialNB(), 100, 10000)

With an accuracy of more then 92%, this model is considerably successful. How does changing parameters such as the number of features, amount of training data, and set of stopwords change the success rate?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

sw = { ',', '\'', '\'\'', '``', '\'\\n', '.', '\\n', '?', '!', '--', '“', '”', ';', ':' }
features = [s for s in range(20,500,20)]
accuracies_test_nb = np.zeros( (2, 2, len(features)) )
for i in range(len(features)) :
    accuracies_test_nb[0][0][i] = test_model(MultinomialNB(), features[i], 10000)
    accuracies_test_nb[0][1][i] = test_model(MultinomialNB(), features[i], 5000)
    accuracies_test_nb[1][0][i] = test_model(MultinomialNB(), features[i], 10000, sw)
    accuracies_test_nb[1][1][i] = test_model(MultinomialNB(), features[i], 5000, sw)
    print("Iteration " + str(i + 1) + " of " + str(len(features)) + " complete")

Let's compare the results with these different parameters.

In [ ]:
plt.plot(features, accuracies_test_nb[0][0], label='Accuracy, test text length of 10000, no stop words')
plt.plot(features, accuracies_test_nb[0][1], label='Accuracy, test text length of 5000, no stop words')
plt.plot(features, accuracies_test_nb[1][0], label='Accuracy, test text length of 10000, with stop words')
plt.plot(features, accuracies_test_nb[1][1], label='Accuracy, test text length of 5000, with stop words')
plt.legend()

Although the models perform similarly, the general trend seems to show that accuracy improves with around 150 features and punctuation removed.

### Multilayer Perceptrons
Neural networks have a vast amount of flexibility which allows them to be specialized to a wide variety of NLP tasks. Multilayer Perceptrons are a simple type of neural network which are easy to use for classification tasks. These may be used with the same type of data as Naive Bayes classifiers, which allows the two approaches to be easily compared.

In [ ]:
accuracies_test_mlp = np.zeros( (2, 2, len(features)) )
for i in range(len(features)) :
    accuracies_test_mlp[0][0][i] = test_model(MLPClassifier(), features[i], 10000)
    accuracies_test_mlp[0][1][i] = test_model(MLPClassifier(), features[i], 5000)
    accuracies_test_mlp[1][0][i] = test_model(MLPClassifier(), features[i], 10000, sw)
    accuracies_test_mlp[1][1][i] = test_model(MLPClassifier(), features[i], 5000, sw)
    print("Iteration " + str(i + 1) + " of " + str(len(features)) + " complete")

How do these results compare with each other and the results from the Naive Bayes classifier?

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(24, 8))

axs[0].plot(features, accuracies_test_nb[0][0], label='NB Accuracy, test text length of 10000, no stop words')
axs[0].plot(features, accuracies_test_nb[0][1], label='NB Accuracy, test text length of 5000, no stop words')
axs[0].plot(features, accuracies_test_nb[1][0], label='NB Accuracy, test text length of 10000, with stop words')
axs[0].plot(features, accuracies_test_nb[1][1], label='NB Accuracy, test text length of 5000, with stop words')
axs[0].legend()

axs[1].plot(features, accuracies_test_mlp[0][0], label='MLP Accuracy, test text length of 10000, no stop words')
axs[1].plot(features, accuracies_test_mlp[0][1], label='MLP Accuracy, test text length of 5000, no stop words')
axs[1].plot(features, accuracies_test_mlp[1][0], label='MLP Accuracy, test text length of 10000, with stop words')
axs[1].plot(features, accuracies_test_mlp[1][1], label='MLP Accuracy, test text length of 5000, with stop words')
axs[1].legend()

Although the results are less consistent, accuracy is still high.

# Conclusions
Based on a comparison of these observations, the following conclusions may be drawn. While the packages in `R.stylo` offer easy-to-interpret output, accuracy with this approach was limited; though PCA had considerable success differentiating Milne's works from Chesterton's and MacDonald's, though the other authors' works were classified more by genre than by authorship, and many techniques yielded similar results.<br>
Naive Bayes Classification was extremely successful, approaching 100% accuracy with well-chosen parameters. Highest rates of accuracy were observed with a number of features around 150 and punctuation removed. The ratio of test to training data did not greatly affect accuracy. Removing punctuation likely aided classification by removing a signal more indicative of genre than authorship; one notes in the lists of features including punctuation that more formal texts tend to feature more formal punctuation, such as by favoring ";" over "--".<br>
Multilayer Perceptrons were also very successful, but less consistently so than methods using Naive Bayes classifiers. As with Naive Bayes, the highest rates of accuracy had punctuation removed, however, the number of features had less effect on the accuracy of the model and the ratio of test data to training data appeared to matter more, with more extensive test data being slightly more important for MLP classifiers. More consistent and accurate models could potentially be created by increasing the number of iterations the model goes through while training.
## Limitations
This study is far from exhaustive, as many other approaches exist for authorship attribution. In addition, the above corpus is relatively small, the number of categories and number of works in each category small, and the number of iterations for training the neural networks far from extensive. Attacking such a problem with a larger corpus and more computing power could yield more exhaustive results. In addition, more extensive sets of stopwords could prove fruitful when using approaches involving `sklearn`; in particular, a tool that removes proper nouns could possibly increase accuracy by removing features like character names. Finally, formatting of the texts in the corpus could also play a role; early approaches were thrown off by the inclusion of Project Gutenberg's license statement's inclusion in every text, which complicated analysis of test data.
## Sources Used
The corpus for this project was drawn from Project Gutenberg's website at www.gutenberg.org. External technologies used for attribution were the `R.stylo` package and Python's `sklearn` package, as well as Python's `matplotlib` package, which was used for creating graphical representation of some data, the `nltk` package, used for word tokenization, and the `numpy` package, used for dealing with matrix data. These were used with reference taken from each respective projects' documentation, as well as course materials. Much of the code for this project was directly inspired by or adapted from that used in the Naive Bayes Classification lab, found in the `nbc.ipynb` Jupyter notebook.